In [44]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names) # organizando em um df
df['target'] = iris.target # rotulos

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train) # treinando o modelo


w = modelo.coef_[0] # pesos do modelo treinado
Vs = X_test.iloc[0].to_dict() # utilizxando a primeira instância dos dados de teste 

delta = []

# Calcula o valor delta para cada feature
# peso negativo, valor menor = maior possibilidade da instância peetencer a classe
# delta c\ peso positivo é (valor feature na instancia - valor min da feature no dataset) * o peso da feature 
# delta c\ peso negativo é (vvalor max da feature no dataset - valor feature na instancia - ) * o peso da feature
for i, feature in enumerate(df.columns[:-1]):  
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - df[feature].min()) * w[i])

# Calcula o limiar (-Gamma_w)

R = sum(delta) # "(Vs predição da regressão loghistica)  # Usando a equação 13 do artigo e garantindo que R seja positivo
# R = O valor absoluto da soma de todos os deltas

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    print(delta)
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True) # enumerando os valores de delta em tuplas e ordenanas
    print(delta_sorted)                                                                               # func lambda recebe a tupla produzida com o valor absoluto em ordem decrescente, do maior para o menor.
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    # Limite para considerar uma feature como importante, com base no limar R
    # threshold_delta = 0 * R # Valor de delta mínimo para ser considerado relevante
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice(soted_idx) e o valor(delta_value) do delta de acordo com o indce(idx)
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação apenas se o delta for maior que o threshold que está sendo calculado com uma % do R. não sei se esta certo isso
       # if abs(delta_value) > threshold_delta:  # Verifica se o delta tem impacto relevante 
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice também para o loop
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

#print(f"Pesos: {w}")
#print(f"Delta: {delta}")
#print(f"Limiar R: {R:.2f}")
#print(f"Vs: {Vs}\n")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")

[0.7088652578943647, 0.7697645172444486, 5.2254240521431345, 1.2983454980344349]
[(2, 5.2254240521431345), (3, 1.2983454980344349), (1, 0.7697645172444486), (0, 0.7088652578943647)]
PI-Explicação: 
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal width (cm) - 2.8
- sepal length (cm) - 6.1


In [4]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # treinando o modelo

# Usa a primeira instância dos dados de teste para a explicação
Vs = X_test.iloc[0].to_dict()  # utilizando a primeira instância dos dados de teste 
instancia_test = X_test.iloc[[0]]  # Mantém como DataFrame para preservar os nomes das características



# Previsão de probabilidades para a instância
# o predict_proba calcula a probabilidade de uma instância pertencer a uma das classes possíveis usando um modelo de classificação treinado.
probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

# O valor de gamma_A é a probabilidade da classe verdadeira
classe_verdadeira = y_test.iloc[0]  # Obtém a classe verdadeira da instância
gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
print(classe_verdadeira)
print(gamma_A)
# Cálculo do valor delta para cada feature
delta = []
w = modelo.coef_[0]  # pesos do modelo treinado

# Calcula o delta para cada feature
for i, feature in enumerate(df.columns[:-1]):
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - df[feature].min()) * w[i])

# Calcula R como a soma dos deltas menos gamma_A
R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)  # enumerando os valores de delta em tuplas e ordenando
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

# Imprime os resultados
print(f"Probabilidades: {probs}")
print(f"Classe verdadeira: {classe_verdadeira}")
print(f"Valor de gamma_A: {gamma_A}")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")


OLHAI MAH -    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
73                6.1               2.8                4.7               1.2
1
0.8277146135745481
Probabilidades: [0.00380009 0.82771461 0.16848529]
Classe verdadeira: 1
Valor de gamma_A: 0.8277146135745481
PI-Explicação: 
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal width (cm) - 2.8
